In [1]:
import json
import pandas as pd
import os
from bs4 import BeautifulSoup
import time
import requests
import statsmodels.api as sm
import numpy as np 

Create Dummy Variables

In [2]:
#Barcelona Dataframes
Barcelona_festival = pd.read_csv("Barcelona_Festival.csv")
#Barcelona_after = pd.read_csv("Barcelona_After.csv")
Barcelona_before = pd.read_csv("Barcelona_before.csv")


# Lisbon Dataframes
Alicante_festival = pd.read_csv("Alicante_festival.csv")
#Alicante_after = pd.read_csv("Alicante_after.csv")
Alicante_before = pd.read_csv("Alicante_before.csv")



In [6]:
# Dummmy Variables Barcelona during 12-16 June 
Barcelona_festival['Treatment_Period'] = 1
Barcelona_festival['Treatment_City'] = 1 

# Dummy Variable Barcelona during 19-23 June 
#Barcelona_after['Treatment_Period'] = 0
#Barcelona_after['Treatment_City'] = 1 

# Dummy Variable Barcelona during 5-9  une 
Barcelona_before['Treatment_Period'] = 0
Barcelona_before['Treatment_City'] = 1 

# Dummy Variable Alicante duing 12-16 June 
Alicante_festival['Treatment_Period'] = 1
Alicante_festival['Treatment_City'] = 0

# Dummy Variable Alicante during 19-23 June 
#Alicante_after['Treatment_Period'] = 0
#Alicante_after['Treatment_City'] = 0

# Dummy Variable Alicante during 5-9 June 
Alicante_before['Treatment_Period'] = 0
Alicante_before['Treatment_City'] = 0


In [7]:
df_total = pd.concat([Barcelona_festival, Barcelona_before, Alicante_festival ,Alicante_before], ignore_index=True)
df_total.head()

,Unnamed: 0,Hotel Name,Rating,Price,Hotel URL,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",€ 1.100,https://www.booking.com/hotel/es/room-mate-ger...,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",€ 1.378,https://www.booking.com/hotel/es/melia-barcelo...,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",€ 799,https://www.booking.com/hotel/es/catalonia-dia...,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",€ 224,https://www.booking.com/hotel/es/casa-nou-barc...,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",€ 964,https://www.booking.com/hotel/es/two-barcelona...,El TWO Hotel Barcelona by Axel está situado en...,1,1


remove the URLs

In [8]:
column_to_remove = ['Hotel URL']
df_total = df_total.drop(columns=column_to_remove)
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",€ 1.100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",€ 1.378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",€ 799,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",€ 224,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",€ 964,El TWO Hotel Barcelona by Axel está situado en...,1,1
...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,€ 1.392,Beautifully presented 5 bedroom villa with WIF...,0,0
3765,885,Villa inaya,"7,9",€ 1.398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0
3766,886,Plaza Toros Apartamento,"4,0",€ 1.656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",€ 1.325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0


In [9]:
# Remove the € from the dataframe 
df_total['Price'] = pd.to_numeric(df_total['Price'].str.replace('€', '').str.replace('.', ''), errors='coerce')
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",1100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",1378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",799,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",224,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",964,El TWO Hotel Barcelona by Axel está situado en...,1,1
...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,1392,Beautifully presented 5 bedroom villa with WIF...,0,0
3765,885,Villa inaya,"7,9",1398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0
3766,886,Plaza Toros Apartamento,"4,0",1656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",1325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0


check the average prices

In [118]:
# Check the median of the prices
average_prices = df_total.groupby(['Treatment_City', 'Treatment_Period'])['Price'].mean()

# If you want to display the result
print(average_prices)

Treatment_City  Treatment_Period
0               0                    634.067492
                1                    676.052511
1               0                   1081.593812
                1                   1480.578842
Name: Price, dtype: float64


In [75]:
# Create a new column for combined treatment condition
#df_total['Treatment_condition'] = df_total['Treatment_Period'] * 2 + df_total['Treatment_City']
#df_total['Price'] = pd.to_numeric(df_total['Price'].str.replace('€', '').str.replace('.', ''), errors='coerce')
#mapping = {0: 'Control_city_p1', 1: 'Treatment_city_p1', 2:'Control_city_p2', 3:'Treatment_city_p2'}
#df_total['Treatment_city_and_treatment'] = df_total['Treatment_condition'].map(mapping)
#df_total = df_total.drop('Treatment_condition', axis = 1)

In [13]:
df_total['Interaction_Period_City'] = df_total['Treatment_Period']  * df_total['Treatment_City']
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City,Interaction_Period_City
0,0,Room Mate Gerard,"8,8",1100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1,1
1,1,Melia Barcelona Sarriá,"8,2",1378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1,1
2,2,Catalonia Diagonal Centro,"8,4",799,El Catalonia Diagonal Centro se encuentra en p...,1,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",224,Barcelona with rooms and lofts in front of the...,1,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",964,El TWO Hotel Barcelona by Axel está situado en...,1,1,1
...,...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,1392,Beautifully presented 5 bedroom villa with WIF...,0,0,0
3765,885,Villa inaya,"7,9",1398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0,0
3766,886,Plaza Toros Apartamento,"4,0",1656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",1325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0,0


In [82]:
#Original column conversion to numerical
#df_total['Price'] = pd.to_numeric(df_total['Price'])

# Dummy Variables conversion to numerical
#df_total['Treatment_Period'] = pd.to_numeric(df_total['Treatment_Period'])
#df_total['Treatment_City'] = pd.to_numeric(df_total['Treatment_City'])
#df_total['Interaction_Period_City'] = pd.to_numeric(df_total['Interaction_Period_City'])

OLS REGRESSIONS

In [10]:
# Regression for Treatment Period 
const_period= sm.add_constant(df_total['Treatment_Period'])
model_per = sm.OLS(df_total['Price'], const_period)
ols_period = model_per.fit()
print(ols_period.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     85.25
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           4.28e-20
Time:                        13:56:52   Log-Likelihood:                -30438.
No. Observations:                3769   AIC:                         6.088e+04
Df Residuals:                    3767   BIC:                         6.089e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              871.2020     17.898  

In [11]:
# Regression for Treatment Period 
const_city= sm.add_constant(df_total['Treatment_City'])
model_city = sm.OLS(df_total['Price'], const_city)
ols_city = model_city.fit()
print(ols_city.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     705.3
Date:                Fri, 02 Feb 2024   Prob (F-statistic):          1.37e-142
Time:                        13:56:57   Log-Likelihood:                -30156.
No. Observations:                3769   AIC:                         6.032e+04
Df Residuals:                    3767   BIC:                         6.033e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            654.9054     17.193     38.

In [14]:
# Regression for Treatment Period 
model_interaction = sm.OLS(df_total['Price'], sm.add_constant(df_total[['Treatment_Period', 'Treatment_City', 'Interaction_Period_City']]))
ols_interaction = model_interaction.fit()
print(ols_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     298.6
Date:                Fri, 02 Feb 2024   Prob (F-statistic):          6.67e-174
Time:                        13:57:15   Log-Likelihood:                -30078.
No. Observations:                3769   AIC:                         6.016e+04
Df Residuals:                    3765   BIC:                         6.019e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     